In [ ]:
from functools import partial
import numpy as np
import torch
import matplotlib.pyplot as plt

alphas = np.linspace(0.1, 1.0)
ratios = np.linspace(0.1, 1.0)
R, A = np.meshgrid(ratios, alphas)
C = 5
b = 0.1
Z = torch.empty((len(ratios), len(alphas)))


def power_constr(ratios, budget, power):
    return (budget / ratios)**(1/power)

def lin_constr(ratios, budget):
    fn = partial(power_constr, power=1)
    return fn(ratios, budget)

def square_constr(ratios, budget):
    fn = partial(power_constr, power=2)
    return fn(ratios, budget)


for i_r, r in enumerate(ratios):
    for i_a, a in enumerate(alphas):
        # Z[i_r, i_a] = r * (C-1) * np.log((1-a)/C) + r*(np.log(a + (1-a)/C)) + C * (1 - r)* np.log(1 / C)
        #Z[i_r, i_a] = r*(np.log(a + (1-a)/C)) + (1 - r)* np.log(1 / C)
        Z[i_r, i_a] = r*(np.log(a + (1-a)/C)) + (1 - r)* np.log(1 / C)
        #Z[i_r, i_a] = r*(np.log((C-1)*a) + 1) + r * np.log(C)

constr = partial(lin_constr, budget = b)
constr = partial(square_constr, budget = b)

fig, ax = plt.subplots()
CS = ax.contour(R, A, Z)
ax.plot(ratios, constr(ratios), 'r--', label="constr")
ax.clabel(CS, inline=True, fontsize=10)
#plt.xscale('log')
ax.set_title("MI approx")
ax.set_xlabel("train set ratio")
ax.set_ylabel("annot qual. alpha")
#ax.legend();
from tikzplotlib import save as tikz_save
tikz_save("closed_form_approx.tikz")

In [ ]:
def f_r(ratios, budget, num_classes):
    return ratios * np.log((num_classes-1)*budget/ratios + 1) + ratios * np.log(num_classes)

In [ ]:
fig, ax = plt.subplots()
ax.plot(ratios, f_r(ratios, b, C), 'r--')
ax.clabel(CS, inline=True, fontsize=10)
#plt.xscale('log')
ax.set_title
ax.set_xlabel("train set ratio")
ax.set_ylabel("annot qual. alpha")